# Semantic Link Labs
Chat GPT request - Can you give the top 15 uses you see people using Semantic Link Labs for? Please provide some code examples as well. Think deeply and look around to see what functions people try to do.

In [ ]:
%pip install -U semantic-link-labs

In [ ]:
import pandas as pd
import sempy_labs as labs
from sempy_labs import admin, report, directlake, lakehouse, graph

In [ ]:
with labs.service_principal_authentication(
    key_vault_uri="https://<your-kv>.vault.azure.net/",
    key_vault_tenant_id="<tenant-id>",
    key_vault_client_id="<client-id>",
    key_vault_client_secret="<client-secret>"
):
    print(admin.list_tenant_settings().head())


### Top 15 uses (with code)

In [ ]:
# 1. Tenant-wide inventory (datasets/reports/workspaces) for documentation & governance

ds = admin.list_datasets()           # all datasets you can see as admin
rp = admin.list_reports()            # all reports
inv = (ds[['Id','Name','WorkspaceName']]
       .merge(rp[['Id','Name','WorkspaceName']], how='outer', left_on='WorkspaceName', right_on='WorkspaceName', suffixes=('_Dataset','_Report')))
inv.head()

# This is the fastest way to answer “what do we have and where?”. 

In [ ]:
# 2. Find & clean up unused artifacts (30+ days no usage)

stale = admin.list_unused_artifacts(workspace="Finance BI")
stale.query("ArtifactType in ['Dataset','Report']").sort_values("LastAccessed")

# Great for license/capacity hygiene. 

In [ ]:
# 3. Access reviews: who can see what (users, groups, SPNs)

ws_access = admin.list_workspace_access_details(workspace="Sales Analytics")
ds_access = admin.list_dataset_users(dataset="Enterprise Model")
user_access = admin.list_access_entities(user_email_address="analyst@contoso.com")

# One-liners for quarterly reviews and audit responses. 

In [ ]:
# 4. Pull audit/activity logs straight into a Lakehouse (security & usage analytics)

from datetime import datetime, timedelta, timezone
end = datetime.now(timezone.utc).replace(microsecond=0)
start = end - timedelta(hours=8)
events = admin.list_activity_events(start_time=start.isoformat(), end_time=end.isoformat(), activity_filter="viewreport")

# This wraps the Power BI Admin Get Activity Events API. 

In [ ]:
# 5. Subscriptions & “widely shared” checks (Published to web / org-wide links)

subs = admin.list_report_subscriptions(report="Executive KPI")
wide = admin.list_widely_shared_artifacts()     # org-wide links & published-to-web

# Quick risk review for external exposure. 

In [ ]:
# 6. Domains & capacity governance (settings overrides, domain workspace listings)

domains = admin.list_domains(non_empty_only=True)
dw = admin.list_domain_workspaces(domain="Brokerage")
cap_overrides = admin.list_capacities_delegated_tenant_settings()

# Helps standardize controls at scale. 

In [ ]:
# 7. Refreshables & refresh history scans (find failing/slow models)

refreshables = admin.get_refreshables()  # tenant or by capacity
refreshables.query("LastRefreshStatus != 'Completed'").head(10)

# Great for proactive SRE-style monitoring. 

In [ ]:
# 8. Workspace documentation (item catalogs) without custom REST plumbing

# e.g., list items per workspace and save as a Delta table
from sempy.fabric import list_items
cat = list_items(workspace="Research Analytics")
cat.to_parquet("Tables/workspace_catalog.parquet")

# Popular for one-click workspace documentation and reporting. 

In [ ]:
# 9 . Rebind reports at scale (e.g., after promoting a new semantic model)

# Rebind ALL reports using one dataset to the new dataset
report.report_rebind_all(
    dataset="Enterprise Model (Test)",
    new_dataset="Enterprise Model (Prod)",
    report_workspace=None   # None = search all workspaces
)

# Handy for environment hops and Direct Lake migrations. 

In [ ]:
# 10. Direct Lake migration helpers (update connections, warm cache, etc.)

from sempy_labs import migration, directlake
directlake.update_direct_lake_model_connection(
    dataset="Trading DL", tables=None, workspace="Ops BI"
)
# …then rebind reports as above

# Labs ships utilities to accelerate Import/DQ → Direct Lake and related housekeeping. 

In [ ]:
# 11. Backup / copy / restore semantic models (PPU/Premium)

# Backup to ADLS Gen2, copy the .abf, then restore in a new workspace
labs.backup_semantic_model(dataset="Enterprise Model", file_path="backup/ent.abf", workspace="Prod")
labs.copy_semantic_model_backup_file(source_workspace="Prod", target_workspace="DR", source_file_name="backup/ent.abf", target_file_name="backup/ent-copy.abf", storage_account="<adls-name>")
labs.restore_semantic_model(dataset="Enterprise Model (DR)", file_path="backup/ent-copy.abf", workspace="DR")

# Great for region moves and DR rehearsals. 

In [ ]:
# 12. Git integration (connect/sync workspaces with GitHub/Azure Repos)

# List Git connections, or connect a workspace to a repo
git_conns = admin.list_git_connections()
# (Connect function available; blog flows show multi-workspace sync patterns)

# Enables repeatable, source-controlled deployments. 

In [ ]:
# 13. Gateway governance & binding (hybrid data sources)

gws = labs.list_gateways()
members = labs.list_gateway_members(gateway=gws.iloc[0]['Id'])
# Bind a dataset to a gateway
labs.bind_dataset_to_gateway(dataset="Ops Model", workspace="Ops BI", gateway=gws.iloc[0]['Id'])

# Inventory gateways, membership, and ensure datasets are bound correctly. 

In [ ]:
# 14. External Data Shares & Managed Private Endpoints (data mesh / B2B)

shares = admin.list_external_data_shares()
# Create/delete Managed Private Endpoint (MPE)
labs.create_workspace_managed_private_endpoint(workspace="Quant", name="mpe-sql", type="Sql", resource_id="<resource-id>")
labs.delete_workspace_managed_private_endpoint(workspace="Quant", name="mpe-sql")

# Useful for governed sharing and private connectivity. 

In [ ]:
# 15. Notify stakeholders via Microsoft Graph (email HTML reports, attach files)

html = "<h3>Unused Artifacts</h3>" + stale.head(10).to_html(index=False)
graph.send_mail(
    user="me@contoso.com",
    subject="[FUAM] Unused artifacts last 30 days",
    to_recipients=["bi-admins@contoso.com"],
    content=html
)

# Graph helpers (including attachments/priority) make it easy to operationalize alerts. 

In [ ]:
A few extra crowd-pleasers

Report surgery: connect to a report’s metadata, list visuals/filters, clone, export/import PBIX/PBIP, and run a Report BPA. Great for repair jobs and standardization. 

TOM operations: add roles/members, perspectives, incremental refresh policies, remove objects — all via Python. Ideal for templating model standards. 

Workspace monitoring (KQL): query the built-in monitoring store or any Eventhouse using query_workspace_monitoring / query_kusto for live SRE-style dashboards. 